# Mega Bar Plot
at the end of this notebook will be one bar plot that will contain experiments his585 and historical from three different models: ESM4, CM4 and CM4x compared to ERA5 data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
%matplotlib inline
%config InlineBackend.figure_format='retina'
import cmocean
import cartopy.crs as ccrs
import matplotlib.path as mpath
import nc_time_axis
import pandas as pd
import matplotlib as mpl
import momlevel as ml
import gcsfs
import zarr
from matplotlib.ticker import FormatStrFormatter
import cftime
import warnings

In [2]:
mask_CM4= xr.open_dataset('/home/pachamama/shared/antarctica_mask/CM4_mask_antarctica.nc')
mask_ESM4= xr.open_dataset('/home/pachamama/shared/antarctica_mask/ESM4_mask_antarctica.nc')
mask_ERA5= xr.open_dataset('/home/pachamama/shared/antarctica_mask/ERA5_mask_antarctica.nc')
mask_CM4X= xr.open_dataset('/home/pachamama/shared/antarctica_mask/CM4X_mask_antarctica.nc')

In [3]:
def standard_grid_cell_area(lat, lon, rE=6371.0e3):
    """ computes the cell area for a standard spherical grid """
    dLat = lat[1] - lat[0]
    dLon = lon[1] - lon[0]
    area = np.empty((len(lat), len(lon)))
    for j in range(0, len(lat)):
        for i in range(0, len(lon)):
            lon1 = lon[i] + dLon / 2
            lon0 = lon[i] - dLon / 2
            lat1 = lat[j] + dLat / 2
            lat0 = lat[j] - dLat / 2
            area[j, i] = (
                (np.pi / 180.0)
                * rE
                * rE
                * np.abs(np.sin(np.radians(lat0)) - np.sin(np.radians(lat1)))
                * np.abs(lon0 - lon1)
            )
    return area

# CM4x historical 

In [4]:
cm4x_uas=xr.open_mfdataset('/backup/tuq41374/model_output/cm4x/p25/historical/atmos/monthly/u_ref/*.nc')
cm4x_uas

<xarray.Dataset>
Dimensions:     (time: 1980, bnds: 2, lat: 360, lon: 576)
Coordinates:
  * bnds        (bnds) float64 1.0 2.0
  * lat         (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon         (lon) float64 0.3125 0.9375 1.562 2.188 ... 358.4 359.1 359.7
  * time        (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
Data variables:
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(60, 360, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(60, 576, 2), meta=np.ndarray>
    time_bnds   (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    u_ref       (time, lat, lon) float32 dask.array<chunksize=(60, 360, 576), meta=np.ndarray>
Attributes:
    filename:              atmos.185001-185412.u_ref.nc
    title:                 CM4_historical_c192_OM4p25
    associated_files:      area: 18540101.grid_spec.nc
    grid_type:             regular
    grid_tile:             N/A
    code_release_version:  19.1
    git_hash:              c97c1f6afab0df3545ee82c337d51accc4205bc3
    creationtime:          Sat Dec 24 23:27:31 2022
    hostname:              pp038
    history:               fregrid --standard_dimension --input_mosaic C192_m...

In [5]:
cm4x_vas=xr.open_mfdataset('/backup/tuq41374/model_output/cm4x/p25/historical/atmos/monthly/v_ref/*.nc')
cm4x_vas

<xarray.Dataset>
Dimensions:     (time: 1980, bnds: 2, lat: 360, lon: 576)
Coordinates:
  * bnds        (bnds) float64 1.0 2.0
  * lat         (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon         (lon) float64 0.3125 0.9375 1.562 2.188 ... 358.4 359.1 359.7
  * time        (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
Data variables:
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(60, 360, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(60, 576, 2), meta=np.ndarray>
    time_bnds   (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    v_ref       (time, lat, lon) float32 dask.array<chunksize=(60, 360, 576), meta=np.ndarray>
Attributes:
    filename:              atmos.185001-185412.v_ref.nc
    title:                 CM4_historical_c192_OM4p25
    associated_files:      area: 18540101.grid_spec.nc
    grid_type:             regular
    grid_tile:             N/A
    code_release_version:  19.1
    git_hash:              c97c1f6afab0df3545ee82c337d51accc4205bc3
    creationtime:          Sat Dec 24 23:27:31 2022
    hostname:              pp038
    history:               fregrid --standard_dimension --input_mosaic C192_m...

In [6]:
cm4x_uas_seasonal=cm4x_uas.u_ref.sel(time=slice('1995-01-16','2014-12-16')).groupby('time.season').mean(dim='time')
cm4x_uas_seasonal

<xarray.DataArray 'u_ref' (season: 4, lat: 360, lon: 576)>
dask.array<stack, shape=(4, 360, 576), dtype=float32, chunksize=(1, 360, 576), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.3125 0.9375 1.562 2.188 ... 357.8 358.4 359.1 359.7
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Attributes:
    long_name:      zonal wind component at 10 m
    units:          m/s
    valid_range:    [-400.  400.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    interp_method:  conserve_order2

In [7]:
cm4x_vas_seasonal=cm4x_vas.v_ref.sel(time=slice('1995-01-16','2014-12-16')).groupby('time.season').mean(dim='time')
cm4x_vas_seasonal

<xarray.DataArray 'v_ref' (season: 4, lat: 360, lon: 576)>
dask.array<stack, shape=(4, 360, 576), dtype=float32, chunksize=(1, 360, 576), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.3125 0.9375 1.562 2.188 ... 357.8 358.4 359.1 359.7
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Attributes:
    long_name:      meridional wind component at 10 m
    units:          m/s
    valid_range:    [-400.  400.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    interp_method:  conserve_order2

In [8]:
cm4x_vas_use=cm4x_vas.v_ref.sel(time=slice('1995-01-16','2014-12-16'))
cm4x_uas_use=cm4x_uas.u_ref.sel(time=slice('1995-01-16','2014-12-16'))

In [9]:
cm4x_his_mergewinds = xr.merge([cm4x_vas_use,cm4x_uas_use]) # merge winds to make it easy
cm4x_his_mergewinds_xmerge = cm4x_his_mergewinds.lon #extracting the lon component fomr the mergewinds array
cm4x_his_mergewinds_ymerge = cm4x_his_mergewinds.lat
cm4x_his_mergewinds_umerge = cm4x_his_mergewinds.u_ref 
cm4x_his_mergewinds_vmerge = cm4x_his_mergewinds.v_ref
cm4x_his_speedmerge = np.sqrt(np.square(cm4x_his_mergewinds_umerge) + np.square(cm4x_his_mergewinds_vmerge)) ## calculating speed using vector formula
## average
cm4x_his_timeaveragemerge = cm4x_his_speedmerge.sel(time=slice('1995-01-16','2014-12-16')).mean(dim='time')

In [10]:
cm4x_his_speedmerge

<xarray.DataArray (time: 240, lat: 360, lon: 576)>
dask.array<sqrt, shape=(240, 360, 576), dtype=float32, chunksize=(60, 360, 576), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.3125 0.9375 1.562 2.188 ... 357.8 358.4 359.1 359.7
  * time     (time) object 1995-01-16 12:00:00 ... 2014-12-16 12:00:00

In [11]:
## merging the UAS seasonal and Vas seasonal to get merge seasonal
cm4x_his_seasonalmerge = xr.merge([cm4x_uas_seasonal,cm4x_vas_seasonal])
cm4x_his_seasonalmerge

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 576, season: 4)
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.3125 0.9375 1.562 2.188 ... 357.8 358.4 359.1 359.7
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Data variables:
    u_ref    (season, lat, lon) float32 dask.array<chunksize=(1, 360, 576), meta=np.ndarray>
    v_ref    (season, lat, lon) float32 dask.array<chunksize=(1, 360, 576), meta=np.ndarray>
Attributes:
    long_name:      zonal wind component at 10 m
    units:          m/s
    valid_range:    [-400.  400.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    interp_method:  conserve_order2

In [12]:
## calculating the seasonal wind speed
cm4x_his_seasonal_speedmerge = np.sqrt(np.square(cm4x_his_seasonalmerge.u_ref) + np.square(cm4x_his_seasonalmerge.v_ref))

## slicing the summer speed
cm4x_his_Summer_szn_speed = cm4x_his_seasonal_speedmerge.sel(season=slice('DJF')).mean(dim='season')
## Slicing summer season from seasonalmerge
cm4x_his_Summer_szn= cm4x_his_seasonalmerge.sel(season='DJF')

## slicing seasonalspeed to get speed for winter and averaging it
cm4x_his_Winter_szn_speed = cm4x_his_seasonal_speedmerge.sel(season=slice('JJA')).mean(dim='season')
cm4x_his_Winter_szn= cm4x_his_seasonalmerge.sel(season='JJA')

## FALL seasonal speed merge and averaging
cm4x_his_Fall_szn_speed = cm4x_his_seasonal_speedmerge.sel(season=slice('MAM')).mean(dim='season')
cm4x_his_Fall_szn= cm4x_his_seasonalmerge.sel(season='MAM')

## slicing seasonalspeed to get spring values and averaging it
cm4x_his_Spring_szn_speed = cm4x_his_seasonal_speedmerge.sel(season=slice('SON')).mean(dim='season')
cm4x_his_Spring_szn= cm4x_his_seasonalmerge.sel(season='SON')

In [13]:
lat_CM4x_historical = cm4x_his_speedmerge.coords['lat'].values #1-D numpy array of model's latitude values
lon_CM4x_historical = cm4x_his_speedmerge.coords['lon'].values #1-D numpy array of model's longitude values
# make sure to replace the name of the data array if you chose a
# different name and use the appropriate coordinate names within the ['']
# based on the coordinates of lat / lon in your dataset.
# Pass the arrays to the standard_grid_cell_area function
areacell = standard_grid_cell_area(lat_CM4x_historical,lon_CM4x_historical)

# The areacella_xarray function outputs a numpy array. Convert this back into
# and xarray DataArray for further use. If this is a field you will be using
# often. You may wish to save this newly created DataArray as a Dataset and save
# as a netcdf file.
# here we assign dimensions and coordinates that are the same as our original SST array:
areacell_CM4x_historical = xr.DataArray(areacell,dims=({'lat':180, 'lon':360}),\
                                                coords=(cm4x_his_speedmerge.coords['lat'],\
                                                        cm4x_his_speedmerge.coords['lon']))

In [14]:
weights_cm4x=areacell_CM4x_historical

In [15]:
weight_ross_CM4x_historical=(mask_CM4X.ross_CM4X*cm4x_his_speedmerge).weighted(weights_cm4x).mean(dim=['lat','lon'])
weight_filchner_CM4x_historical=(mask_CM4X.filchner_CM4X*cm4x_his_speedmerge).weighted(weights_cm4x).mean(dim=['lat','lon'])
weight_peninsula_CM4x_historical= (mask_CM4X.peninsula_CM4X*cm4x_his_speedmerge).weighted(weights_cm4x).mean(dim=['lat','lon'])
weight_loweast_CM4x_historical= (mask_CM4X.loweast_CM4X*cm4x_his_speedmerge).weighted(weights_cm4x).mean(dim=['lat','lon'])
weight_lowwest_CM4x_historical= (mask_CM4X.lowwest_CM4X*cm4x_his_speedmerge).weighted(weights_cm4x).mean(dim=['lat','lon'])
weight_plateau_CM4x_historical= (mask_CM4X.plateau_CM4X*cm4x_his_speedmerge).weighted(weights_cm4x).mean(dim=['lat','lon'])

In [16]:
weight_ross_cm4x_annual_historical = ml.util.annual_average(weight_ross_CM4x_historical,tcoord='time')
weight_filchner_cm4x_annual_historical= ml.util.annual_average(weight_filchner_CM4x_historical, tcoord='time')
weight_peninsula_cm4x_annual_historical=ml.util.annual_average(weight_peninsula_CM4x_historical, tcoord='time')
weight_loweast_cm4x_annual_historical=ml.util.annual_average(weight_loweast_CM4x_historical, tcoord='time')
weight_lowwest_cm4x_annual_historical=ml.util.annual_average(weight_lowwest_CM4x_historical, tcoord='time')
weight_plateau_cm4x_annual_historical=ml.util.annual_average(weight_plateau_CM4x_historical, tcoord='time')

In [17]:
cm4x_his_ross_wind=weight_ross_cm4x_annual_historical.mean(dim='time')
cm4x_his_filch_wind=weight_filchner_cm4x_annual_historical.mean(dim='time')
cm4x_his_pen_wind=weight_peninsula_cm4x_annual_historical.mean(dim='time')
cm4x_his_east_wind=weight_loweast_cm4x_annual_historical.mean(dim='time')
cm4x_his_west_wind=weight_lowwest_cm4x_annual_historical.mean(dim='time')
cm4x_his_plat_wind=weight_plateau_cm4x_annual_historical.mean(dim='time')

In [18]:
print(cm4x_his_ross_wind.values)
print(cm4x_his_filch_wind.values)
print(cm4x_his_pen_wind.values)
print(cm4x_his_east_wind.values)
print(cm4x_his_west_wind.values)
print(cm4x_his_plat_wind.values)

2.795495161565733
2.4551558437181096
3.046842730252119
5.91763074811499
4.067977060422126
4.6643710997910235


In [19]:
ross_total_std_CM4x= weight_ross_cm4x_annual_historical.std('time')
filchner_total_std_CM4x= weight_filchner_cm4x_annual_historical.std('time')
peninsula_total_std_CM4x=weight_peninsula_cm4x_annual_historical.std('time')
loweast_total_std_CM4x=weight_loweast_cm4x_annual_historical.std('time')
lowwest_total_std_CM4x=weight_lowwest_cm4x_annual_historical.std('time')
plateau_total_std_CM4x=weight_plateau_cm4x_annual_historical.std('time')

In [20]:
print(ross_total_std_CM4x.values)
print(filchner_total_std_CM4x.values)
print(peninsula_total_std_CM4x.values)
print(loweast_total_std_CM4x.values)
print(lowwest_total_std_CM4x.values)
print(plateau_total_std_CM4x.values)

0.08563159452774731
0.09880345534111222
0.11616729623324284
0.04899005280025134
0.06580244510510012
0.06702133318191811


In [21]:
ross_total_CM4X_his= weight_ross_CM4x_historical.mean('time')
filchner_total_CM4X_his=weight_filchner_CM4x_historical.mean('time')
peninsula_total_CM4X_his=weight_peninsula_CM4x_historical.mean('time')
loweast_total_CM4X_his=weight_loweast_CM4x_historical.mean('time')
lowwest_total_CM4X_his=weight_lowwest_CM4x_historical.mean('time')
plateau_total_CM4X_his=weight_plateau_CM4x_historical.mean('time')

In [22]:
ross_total_val_CM4X_his= print(ross_total_CM4X_his.values)
filchner_total_val_CM4X_his=print(filchner_total_CM4X_his.values)
peninsula_total_val_CM4X_his=print(peninsula_total_CM4X_his.values)
loweast_total_val_CM4X_his=print(loweast_total_CM4X_his.values)
lowwest_total_val_CM4X_his=print(lowwest_total_CM4X_his.values)
plateau_total_val_CM4X_his=print(plateau_total_CM4X_his.values)

2.7948578072203487
2.4545504671235236
3.046108256536738
5.913807171022635
4.066487088117113
4.661868222546543


# CM4x SSP585

In [23]:
cm4x_uas_ssp585=xr.open_mfdataset('/backup/tuq41374/model_output/cm4x/p25/ssp585/atmos/monthly/u_ref/*.nc')
cm4x_uas_ssp585

<xarray.Dataset>
Dimensions:     (time: 1020, bnds: 2, lat: 360, lon: 576)
Coordinates:
  * bnds        (bnds) float64 1.0 2.0
  * lat         (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon         (lon) float64 0.3125 0.9375 1.562 2.188 ... 358.4 359.1 359.7
  * time        (time) object 2015-01-16 12:00:00 ... 2099-12-16 12:00:00
Data variables:
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(60, 360, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(60, 576, 2), meta=np.ndarray>
    time_bnds   (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    u_ref       (time, lat, lon) float32 dask.array<chunksize=(60, 360, 576), meta=np.ndarray>
Attributes:
    filename:              atmos.201501-201912.u_ref.nc
    title:                 CM4_ssp585_c192_OM4p25
    associated_files:      area: 20190101.grid_spec.nc
    grid_type:             regular
    grid_tile:             N/A
    code_release_version:  19.1
    git_hash:              c97c1f6afab0df3545ee82c337d51accc4205bc3
    creationtime:          Thu Feb 16 13:17:56 2023
    hostname:              pp033
    history:               fregrid --standard_dimension --input_mosaic C192_m...

In [24]:
cm4x_vas_ssp585=xr.open_mfdataset('/backup/tuq41374/model_output/cm4x/p25/ssp585/atmos/monthly/v_ref/*.nc')
cm4x_vas_ssp585

<xarray.Dataset>
Dimensions:     (time: 1020, bnds: 2, lat: 360, lon: 576)
Coordinates:
  * bnds        (bnds) float64 1.0 2.0
  * lat         (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon         (lon) float64 0.3125 0.9375 1.562 2.188 ... 358.4 359.1 359.7
  * time        (time) object 2015-01-16 12:00:00 ... 2099-12-16 12:00:00
Data variables:
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(60, 360, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(60, 576, 2), meta=np.ndarray>
    time_bnds   (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    v_ref       (time, lat, lon) float32 dask.array<chunksize=(60, 360, 576), meta=np.ndarray>
Attributes:
    filename:              atmos.201501-201912.v_ref.nc
    title:                 CM4_ssp585_c192_OM4p25
    associated_files:      area: 20190101.grid_spec.nc
    grid_type:             regular
    grid_tile:             N/A
    code_release_version:  19.1
    git_hash:              c97c1f6afab0df3545ee82c337d51accc4205bc3
    creationtime:          Thu Feb 16 13:17:56 2023
    hostname:              pp033
    history:               fregrid --standard_dimension --input_mosaic C192_m...

In [25]:
cm4x_uas_ssp_seasonal=cm4x_uas_ssp585.u_ref.sel(time=slice('2081-01-16', '2100-12-16')).groupby('time.season').mean(dim='time')
cm4x_vas_ssp_seasonal=cm4x_vas_ssp585.v_ref.sel(time=slice('2081-01-16', '2100-12-16')).groupby('time.season').mean(dim='time')
cm4x_vas_ssp_seasonal

<xarray.DataArray 'v_ref' (season: 4, lat: 360, lon: 576)>
dask.array<stack, shape=(4, 360, 576), dtype=float32, chunksize=(1, 360, 576), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.3125 0.9375 1.562 2.188 ... 357.8 358.4 359.1 359.7
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Attributes:
    long_name:      meridional wind component at 10 m
    units:          m/s
    valid_range:    [-400.  400.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    interp_method:  conserve_order2

In [26]:
cm4x_ssp_uas_use=cm4x_uas_ssp585.sel(time=slice('2081-01-16', '2100-12-16'))
cm4x_ssp_vas_use=cm4x_vas_ssp585.sel(time=slice('2081-01-16', '2100-12-16'))

In [27]:
cm4x_ssp_mergewinds = xr.merge([cm4x_ssp_uas_use,cm4x_ssp_vas_use]) # merge winds to make it easy
cm4x_ssp_mergewinds_xmerge = cm4x_ssp_mergewinds.lon #extracting the lon component fomr the mergewinds array
cm4x_ssp_mergewinds_ymerge = cm4x_ssp_mergewinds.lat
cm4x_ssp_mergewinds_umerge = cm4x_ssp_mergewinds.u_ref 
cm4x_ssp_mergewinds_vmerge = cm4x_ssp_mergewinds.v_ref
cm4x_ssp_speedmerge = np.sqrt(np.square(cm4x_ssp_mergewinds_umerge) + np.square(cm4x_ssp_mergewinds_vmerge)) ## calculating speed using vector formula
## average
cm4x_ssp_timeaveragemerge = cm4x_ssp_speedmerge.sel(time=slice('2081-01-16', '2100-12-16')).mean(dim='time')

In [28]:
cm4x_ssp_speedmerge

<xarray.DataArray (time: 228, lat: 360, lon: 576)>
dask.array<sqrt, shape=(228, 360, 576), dtype=float32, chunksize=(60, 360, 576), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.3125 0.9375 1.562 2.188 ... 357.8 358.4 359.1 359.7
  * time     (time) object 2081-01-16 12:00:00 ... 2099-12-16 12:00:00

In [29]:
cm4x_ssp_seasonal_merge=xr.merge([cm4x_uas_ssp_seasonal,cm4x_vas_ssp_seasonal])
cm4x_ssp_seasonal_merge

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 576, season: 4)
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.3125 0.9375 1.562 2.188 ... 357.8 358.4 359.1 359.7
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Data variables:
    u_ref    (season, lat, lon) float32 dask.array<chunksize=(1, 360, 576), meta=np.ndarray>
    v_ref    (season, lat, lon) float32 dask.array<chunksize=(1, 360, 576), meta=np.ndarray>
Attributes:
    long_name:      zonal wind component at 10 m
    units:          m/s
    valid_range:    [-400.  400.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    interp_method:  conserve_order2

In [30]:
## calculating the seasonal wind speed
cm4x_ssp_seasonal_speedmerge = np.sqrt(np.square(cm4x_ssp_seasonal_merge.u_ref) + np.square(cm4x_ssp_seasonal_merge.v_ref))

## slicing the summer speed
cm4x_ssp_Summer_szn_speed = cm4x_ssp_seasonal_speedmerge.sel(season=slice('DJF')).mean(dim='season')
## Slicing summer season from seasonalmerge
cm4x_ssp_Summer_szn= cm4x_ssp_seasonal_merge.sel(season='DJF')

## slicing seasonalspeed to get speed for winter and averaging it
cm4x_ssp_Winter_szn_speed = cm4x_ssp_seasonal_speedmerge.sel(season=slice('JJA')).mean(dim='season')
cm4x_ssp_Winter_szn= cm4x_ssp_seasonal_merge.sel(season='JJA')

## FALL seasonal speed merge and averaging
cm4x_ssp_Fall_szn_speed = cm4x_ssp_seasonal_speedmerge.sel(season=slice('MAM')).mean(dim='season')
cm4x_ssp_Fall_szn= cm4x_ssp_seasonal_merge.sel(season='MAM')

## slicing seasonalspeed to get spring values and averaging it
cm4x_ssp_Spring_szn_speed = cm4x_ssp_seasonal_speedmerge.sel(season=slice('SON')).mean(dim='season')
cm4x_ssp_Spring_szn= cm4x_ssp_seasonal_merge.sel(season='SON')

In [31]:
lat_CM4x_ssp = cm4x_ssp_speedmerge.coords['lat'].values #1-D numpy array of model's latitude values
lon_CM4x_ssp = cm4x_ssp_speedmerge.coords['lon'].values #1-D numpy array of model's longitude values
# make sure to replace the name of the data array if you chose a
# different name and use the appropriate coordinate names within the ['']
# based on the coordinates of lat / lon in your dataset.
# Pass the arrays to the standard_grid_cell_area function
areacell = standard_grid_cell_area(lat_CM4x_ssp,lon_CM4x_ssp)

# The areacella_xarray function outputs a numpy array. Convert this back into
# and xarray DataArray for further use. If this is a field you will be using
# often. You may wish to save this newly created DataArray as a Dataset and save
# as a netcdf file.
# here we assign dimensions and coordinates that are the same as our original SST array:
areacell_CM4x_ssp = xr.DataArray(areacell,dims=({'lat':180, 'lon':360}),\
                                                coords=(cm4x_ssp_speedmerge.coords['lat'],\
                                                        cm4x_ssp_speedmerge.coords['lon']))

In [32]:
ssp_weights_cm4x=areacell_CM4x_ssp

In [33]:
weight_ross_CM4x_ssp=(mask_CM4X.ross_CM4X*cm4x_ssp_speedmerge).weighted(ssp_weights_cm4x).mean(dim=['lat','lon'])
weight_filchner_CM4x_ssp=(mask_CM4X.filchner_CM4X*cm4x_ssp_speedmerge).weighted(ssp_weights_cm4x).mean(dim=['lat','lon'])
weight_peninsula_CM4x_ssp= (mask_CM4X.peninsula_CM4X*cm4x_ssp_speedmerge).weighted(ssp_weights_cm4x).mean(dim=['lat','lon'])
weight_loweast_CM4x_ssp= (mask_CM4X.loweast_CM4X*cm4x_ssp_speedmerge).weighted(ssp_weights_cm4x).mean(dim=['lat','lon'])
weight_lowwest_CM4x_ssp= (mask_CM4X.lowwest_CM4X*cm4x_ssp_speedmerge).weighted(ssp_weights_cm4x).mean(dim=['lat','lon'])
weight_plateau_CM4x_ssp= (mask_CM4X.plateau_CM4X*cm4x_ssp_speedmerge).weighted(ssp_weights_cm4x).mean(dim=['lat','lon'])

In [34]:
weight_ross_cm4x_annual_ssp = ml.util.annual_average(weight_ross_CM4x_ssp,tcoord='time')
weight_filchner_cm4x_annual_ssp= ml.util.annual_average(weight_filchner_CM4x_ssp, tcoord='time')
weight_peninsula_cm4x_annual_ssp=ml.util.annual_average(weight_peninsula_CM4x_ssp, tcoord='time')
weight_loweast_cm4x_annual_ssp=ml.util.annual_average(weight_loweast_CM4x_ssp, tcoord='time')
weight_lowwest_cm4x_annual_ssp=ml.util.annual_average(weight_lowwest_CM4x_ssp, tcoord='time')
weight_plateau_cm4x_annual_ssp=ml.util.annual_average(weight_plateau_CM4x_ssp, tcoord='time')

In [35]:
cm4x_ssp_ross_wind=weight_ross_cm4x_annual_ssp.mean(dim='time')
cm4x_ssp_filch_wind=weight_filchner_cm4x_annual_ssp.mean(dim='time')
cm4x_ssp_pen_wind=weight_peninsula_cm4x_annual_ssp.mean(dim='time')
cm4x_ssp_east_wind=weight_loweast_cm4x_annual_ssp.mean(dim='time')
cm4x_ssp_west_wind=weight_lowwest_cm4x_annual_ssp.mean(dim='time')
cm4x_ssp_plat_wind=weight_plateau_cm4x_annual_ssp.mean(dim='time')

In [36]:
print(cm4x_ssp_ross_wind.values)
print(cm4x_ssp_filch_wind.values)
print(cm4x_ssp_pen_wind.values)
print(cm4x_ssp_east_wind.values)
print(cm4x_ssp_west_wind.values)
print(cm4x_ssp_plat_wind.values)

2.6883911524608126
2.4005182851131757
3.011193493951181
5.726419567260154
3.954811912334859
4.571322007188408


In [37]:
ross_total_std_CM4x_ssp= weight_ross_cm4x_annual_ssp.std('time')
filchner_total_std_CM4x_ssp= weight_filchner_cm4x_annual_ssp.std('time')
peninsula_total_std_CM4x_ssp=weight_peninsula_cm4x_annual_ssp.std('time')
loweast_total_std_CM4x_ssp=weight_loweast_cm4x_annual_ssp.std('time')
lowwest_total_std_CM4x_ssp=weight_lowwest_cm4x_annual_ssp.std('time')
plateau_total_std_CM4x_ssp=weight_plateau_cm4x_annual_ssp.std('time')

In [38]:
print(ross_total_std_CM4x_ssp.values)
print(filchner_total_std_CM4x_ssp.values)
print(peninsula_total_std_CM4x_ssp.values)
print(loweast_total_std_CM4x_ssp.values)
print(lowwest_total_std_CM4x_ssp.values)
print(plateau_total_std_CM4x_ssp.values)

0.12284717596089571
0.08617890230707151
0.20461534667324466
0.07259554082602682
0.09949711868152591
0.06288855821790354


# ESM4 historical 

In [39]:
esm4 = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
#defines the Eastward Near-Surface Wind
gfdl_esm4_uas_data = esm4.query(
    "table_id == 'Amon' & \
    variable_id == 'uas' & \
    experiment_id == 'historical' & \
    institution_id == 'NOAA-GFDL' &  \
    source_id == 'GFDL-ESM4'")

gcs = gcsfs.GCSFileSystem(token='anon')
zstore_esm4_uas = gfdl_esm4_uas_data.zstore.values[-1]
mapper_esm4_uas = gcs.get_mapper(zstore_esm4_uas)

esm4_uas = xr.open_zarr(mapper_esm4_uas, 
                                consolidated=True,use_cftime=True)

#defines the NorthWard Near-Surface Wind
gfdl_esm4_vas_data = esm4.query(
    "table_id == 'Amon' & \
    variable_id == 'vas' & \
    experiment_id == 'historical' & \
    institution_id == 'NOAA-GFDL' &  \
    source_id == 'GFDL-ESM4'")

zstore_esm4_vas = gfdl_esm4_vas_data.zstore.values[-1]
mapper_esm4_vas = gcs.get_mapper(zstore_esm4_vas)

esm4_vas = xr.open_zarr(mapper_esm4_vas, 
                                consolidated=True,use_cftime=True)

In [40]:
esm4_vas

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 288, time: 1980)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
    height     float64 ...
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(180, 2), meta=np.ndarray>
  * lon        (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(288, 2), meta=np.ndarray>
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
Data variables:
    vas        (time, lat, lon) float32 dask.array<chunksize=(596, 180, 288), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  36500.0
    comment:                <null ref>
    ...                     ...
    tracking_id:            hdl:21.14100/39b6212a-af33-4131-b50b-7b10c22594ab...
    variable_id:            vas
    variant_info:           N/A
    variant_label:          r1i1p1f1
    netcdf_tracking_ids:    hdl:21.14100/39b6212a-af33-4131-b50b-7b10c22594ab...
    version_id:             v20190726

In [41]:
uase4=esm4_uas.uas.sel(time=slice('1995-01-16','2014-12-16')) #eastward wind variable
## grouping the data into seasons and averaging the seasons
ESM4_uas_seasonal = uase4.groupby('time.season').mean(dim='time')
ESM4_uas_seasonal

## Slicing the eastward wind into the south pole region
vase4 = esm4_vas.vas.sel(time=slice('1995-01-16','2014-12-16')) #northward wind variable
## grouping the data into seasons and averaging the 500 seasons
ESM4_vas_seasonal = vase4.groupby('time.season').mean(dim='time')
ESM4_vas_seasonal

<xarray.DataArray 'vas' (season: 4, lat: 180, lon: 288)>
dask.array<stack, shape=(4, 180, 288), dtype=float32, chunksize=(1, 180, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 ...
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    interp_method:  conserve_order2
    long_name:      Northward Near-Surface Wind
    original_name:  vas
    standard_name:  northward_wind
    units:          m s-1

In [42]:
mergedwindse4 = xr.merge([uase4,vase4])
esm4_xmerge=mergedwindse4.lon
esm4_ymerge=mergedwindse4.lat
esm4_umerge=mergedwindse4.uas
esm4_vmerge=mergedwindse4.vas
esm4_speed_merge=np.sqrt(np.square(esm4_umerge) + np.square(esm4_vmerge))
esm4_speed_merge
#wind speed around antarctica 

esm4_timeaveragemerge = mergedwindse4.sel(time=slice('1995-01-16','2014-12-16')).mean(dim='time')
esm4_timeaveragemerge

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 288)
Coordinates:
    height   float64 10.0
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
Data variables:
    uas      (lat, lon) float32 dask.array<chunksize=(180, 288), meta=np.ndarray>
    vas      (lat, lon) float32 dask.array<chunksize=(180, 288), meta=np.ndarray>

In [43]:
ESM4_speedslice = esm4_speed_merge.mean(dim='time')
ESM4_speedslice

<xarray.DataArray (lat: 180, lon: 288)>
dask.array<mean_agg-aggregate, shape=(180, 288), dtype=float32, chunksize=(180, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 10.0
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4

In [44]:
#ESM4_vas_seasonal defined earlier
## merging the UAS seasonal and Vas seasonal to get merge seasonal
esm4_seasonalmerge= xr.merge([ESM4_uas_seasonal,ESM4_vas_seasonal])

##
## calculating the seasonal wind speed
esm4_SPEED_seasonalmerge = np.sqrt(np.square(esm4_seasonalmerge.uas) + np.square(esm4_seasonalmerge.vas))


In [45]:
## slicing the summer speed
ESM4_Summer_szn_speed = esm4_SPEED_seasonalmerge.sel(season=slice('DJF')).mean(dim='season')
## Slicing summer season from seasonalmerge
ESM4_Summer_szn= esm4_seasonalmerge.sel(season='DJF') #MUST USE SEASONAL MERGE NOT SPEED SEASONAL MERGE. BE CAREFUL

## slicing the winter speed
ESM4_winter_szn_speed = esm4_SPEED_seasonalmerge.sel(season=slice('JJA')).mean(dim='season')
## Slicing winter season from seasonalmerge
ESM4_winter_szn= esm4_seasonalmerge.sel(season='JJA') #MUST USE SEASONAL MERGE NOT SPEED SEASONAL MERGE. BE CAREFUL

## slicing the fall speed
ESM4_fall_szn_speed = esm4_SPEED_seasonalmerge.sel(season=slice('MAM')).mean(dim='season')
## Slicing fall season from seasonalmerge
ESM4_fall_szn= esm4_seasonalmerge.sel(season='MAM') #MUST USE SEASONAL MERGE NOT SPEED SEASONAL MERGE. BE CAREFUL

## slicing the spring speed
ESM4_spring_szn_speed = esm4_SPEED_seasonalmerge.sel(season=slice('SON')).mean(dim='season')
## Slicing spring season from seasonalmerge
ESM4_spring_szn= esm4_seasonalmerge.sel(season='SON') #MUST USE SEASONAL MERGE NOT SPEED SEASONAL MERGE. BE CAREFUL

In [46]:
lat_esm4_historical = esm4_speed_merge.coords['lat'].values #1-D numpy array of model's latitude values
lon_esm4_historical = esm4_speed_merge.coords['lon'].values #1-D numpy array of model's longitude values
# make sure to replace the name of the data array if you chose a
# different name and use the appropriate coordinate names within the ['']
# based on the coordinates of lat / lon in your dataset.
# Pass the arrays to the standard_grid_cell_area function
esm4_his_areacell = standard_grid_cell_area(lat_esm4_historical,lon_esm4_historical)

# The areacella_xarray function outputs a numpy array. Convert this back into
# and xarray DataArray for further use. If this is a field you will be using
# often. You may wish to save this newly created DataArray as a Dataset and save
# as a netcdf file.
# here we assign dimensions and coordinates that are the same as our original SST array:
areacell_esm4_historical = xr.DataArray(esm4_his_areacell,dims=({'lat':180, 'lon':360}),\
                                                coords=(esm4_speed_merge.coords['lat'],\
                                                        esm4_speed_merge.coords['lon']))

In [47]:
weights_esm4_his=areacell_esm4_historical

In [48]:
weight_ross_ESM4_historical=(mask_ESM4.ross_ESM4*esm4_speed_merge).weighted(weights_esm4_his).mean(dim=['lat','lon'])
weight_filchner_ESM4_historical=(mask_ESM4.filchner_ESM4*esm4_speed_merge).weighted(weights_esm4_his).mean(dim=['lat','lon'])
weight_peninsula_ESM4_historical= (mask_ESM4.peninsula_ESM4*esm4_speed_merge).weighted(weights_esm4_his).mean(dim=['lat','lon'])
weight_loweast_ESM4_historical= (mask_ESM4.loweast_ESM4*esm4_speed_merge).weighted(weights_esm4_his).mean(dim=['lat','lon'])
weight_lowwest_ESM4_historical= (mask_ESM4.lowwest_ESM4*esm4_speed_merge).weighted(weights_esm4_his).mean(dim=['lat','lon'])
weight_plateau_ESM4_historical= (mask_ESM4.plateau_ESM4*esm4_speed_merge).weighted(weights_esm4_his).mean(dim=['lat','lon'])

In [49]:
weight_ross_ESM4_annual_historical = ml.util.annual_average(weight_ross_ESM4_historical,tcoord='time')
weight_filchner_ESM4_annual_historical= ml.util.annual_average(weight_filchner_ESM4_historical, tcoord='time')
weight_peninsula_ESM4_annual_historical=ml.util.annual_average(weight_peninsula_ESM4_historical, tcoord='time')
weight_loweast_ESM4_annual_historical=ml.util.annual_average(weight_loweast_ESM4_historical, tcoord='time')
weight_lowwest_ESM4_annual_historical=ml.util.annual_average(weight_lowwest_ESM4_historical, tcoord='time')
weight_plateau_ESM4_annual_historical=ml.util.annual_average(weight_plateau_ESM4_historical, tcoord='time')

In [50]:
ESM4_historical_ross_wind=weight_ross_ESM4_annual_historical.mean(dim='time')
ESM4_historical_filch_wind=weight_filchner_ESM4_annual_historical.mean(dim='time')
ESM4_historical_pen_wind=weight_peninsula_ESM4_annual_historical.mean(dim='time')
ESM4_historical_east_wind=weight_loweast_ESM4_annual_historical.mean(dim='time')
ESM4_historical_west_wind=weight_lowwest_ESM4_annual_historical.mean(dim='time')
ESM4_historical_plat_wind=weight_plateau_ESM4_annual_historical.mean(dim='time')

In [51]:
print(ESM4_historical_ross_wind.values)
print(ESM4_historical_filch_wind.values)
print(ESM4_historical_pen_wind.values)
print(ESM4_historical_east_wind.values)
print(ESM4_historical_west_wind.values)
print(ESM4_historical_plat_wind.values)

2.717323193980317
2.413653711220919
3.2008522669365895
6.944507789949228
4.556693396811866
4.876263966612993


In [52]:
ross_total_std_ESM4= weight_ross_ESM4_annual_historical.std('time')
filchner_total_std_ESM4= weight_filchner_ESM4_annual_historical.std('time')
peninsula_total_std_ESM4=weight_peninsula_ESM4_annual_historical.std('time')
loweast_total_std_ESM4=weight_loweast_ESM4_annual_historical.std('time')
lowwest_total_std_ESM4=weight_lowwest_ESM4_annual_historical.std('time')
plateau_total_std_ESM4=weight_plateau_ESM4_annual_historical.std('time')

In [53]:
print(ross_total_std_ESM4.values)
print(filchner_total_std_ESM4.values)
print(peninsula_total_std_ESM4.values)
print(loweast_total_std_ESM4.values)
print(lowwest_total_std_ESM4.values)
print(plateau_total_std_ESM4.values)

0.12827888028683546
0.1112760032392988
0.1301968465953218
0.0924209210316265
0.10579032034074715
0.09259222616016305


# ESM4 ssp585

In [54]:
google_cloud_cmip6 = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
#defines the Eastward Near-Surface Wind
gfdl_esm4_uas_ssp_data = google_cloud_cmip6.query(
    "table_id == 'Amon' & \
    variable_id == 'uas' & \
    experiment_id == 'ssp585' & \
    institution_id == 'NOAA-GFDL' &  \
    source_id == 'GFDL-ESM4'")

gcs = gcsfs.GCSFileSystem(token='anon')
zstore_esm4_uas = gfdl_esm4_uas_ssp_data.zstore.values[-1]
mapper_esm4_uas = gcs.get_mapper(zstore_esm4_uas)

esm4_uas_ssp = xr.open_zarr(mapper_esm4_uas, 
                                consolidated=True,use_cftime=True).uas
esm4_uas_ssp

<xarray.DataArray 'uas' (time: 1032, lat: 180, lon: 288)>
dask.array<open_dataset-uas, shape=(1032, 180, 288), dtype=float32, chunksize=(613, 180, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 ...
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * time     (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    interp_method:  conserve_order2
    long_name:      Eastward Near-Surface Wind
    original_name:  uas
    standard_name:  eastward_wind
    units:          m s-1

In [55]:
#defines the NorthWard Near-Surface Wind
gfdl_esm4_vas_ssp_data = google_cloud_cmip6.query(
    "table_id == 'Amon' & \
    variable_id == 'vas' & \
    experiment_id == 'ssp585' & \
    institution_id == 'NOAA-GFDL' &  \
    source_id == 'GFDL-ESM4'")

zstore_esm4_vas = gfdl_esm4_vas_ssp_data.zstore.values[-1]
mapper_esm4_vas = gcs.get_mapper(zstore_esm4_vas)

esm4_vas_ssp = xr.open_zarr(mapper_esm4_vas, 
                                consolidated=True,use_cftime=True).vas
esm4_vas_ssp

<xarray.DataArray 'vas' (time: 1032, lat: 180, lon: 288)>
dask.array<open_dataset-vas, shape=(1032, 180, 288), dtype=float32, chunksize=(596, 180, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 ...
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * time     (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    interp_method:  conserve_order2
    long_name:      Northward Near-Surface Wind
    original_name:  vas
    standard_name:  northward_wind
    units:          m s-1

In [56]:
#eastward wind variable 
uase4_ssp=esm4_uas_ssp.sel(time=slice('2081-01-16','2100-12-16'))
## grouping the data into seasons and averaging the seasons
ESM4_uas_ssp_seasonal = uase4_ssp.groupby('time.season').mean(dim='time')
ESM4_uas_ssp_seasonal

<xarray.DataArray 'uas' (season: 4, lat: 180, lon: 288)>
dask.array<stack, shape=(4, 180, 288), dtype=float32, chunksize=(1, 180, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 ...
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    interp_method:  conserve_order2
    long_name:      Eastward Near-Surface Wind
    original_name:  uas
    standard_name:  eastward_wind
    units:          m s-1

In [57]:
## Slicing the eastward wind into the south pole region
vase4_ssp = esm4_vas_ssp.sel(time=slice('2081-01-16','2100-12-16'))#northward wind variable
## grouping the data into seasons and averaging the 500 seasons
ESM4_vas_ssp_seasonal = vase4.groupby('time.season').mean(dim='time')
ESM4_vas_ssp_seasonal

<xarray.DataArray 'vas' (season: 4, lat: 180, lon: 288)>
dask.array<stack, shape=(4, 180, 288), dtype=float32, chunksize=(1, 180, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 ...
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    interp_method:  conserve_order2
    long_name:      Northward Near-Surface Wind
    original_name:  vas
    standard_name:  northward_wind
    units:          m s-1

In [58]:
ssp_mergedwindse4 = xr.merge([uase4_ssp,vase4_ssp])
ssp_esm4_xmerge=ssp_mergedwindse4.lon
ssp_esm4_ymerge=ssp_mergedwindse4.lat
ssp_esm4_umerge=ssp_mergedwindse4.uas
ssp_esm4_vmerge=ssp_mergedwindse4.vas
ssp_esm4_speed_merge=np.sqrt(np.square(ssp_esm4_umerge) + np.square(ssp_esm4_vmerge))
ssp_esm4_speed_merge
#wind speed around antarctica 

ssp_esm4_timeaveragemerge = mergedwindse4.sel(time=slice('2081-01-16','2100-12-16')).mean(dim='time')
ssp_esm4_timeaveragemerge

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 288)
Coordinates:
    height   float64 10.0
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
Data variables:
    uas      (lat, lon) float32 dask.array<chunksize=(180, 288), meta=np.ndarray>
    vas      (lat, lon) float32 dask.array<chunksize=(180, 288), meta=np.ndarray>

In [59]:
## merging the UAS seasonal and Vas seasonal to get merge seasonal
ssp_esm4_seasonalmerge= xr.merge([ESM4_uas_ssp_seasonal,ESM4_vas_ssp_seasonal])

##
## calculating the seasonal wind speed
ssp_esm4_SPEED_seasonalmerge = np.sqrt(np.square(ssp_esm4_seasonalmerge.uas) + np.square(ssp_esm4_seasonalmerge.vas))
ssp_esm4_SPEED_seasonalmerge

<xarray.DataArray (season: 4, lat: 180, lon: 288)>
dask.array<sqrt, shape=(4, 180, 288), dtype=float32, chunksize=(1, 180, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 10.0
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'

In [60]:
## slicing the summer speed
ssp_ESM4_Summer_szn_speed = ssp_esm4_SPEED_seasonalmerge.sel(season=slice('DJF')).mean(dim='season')
## Slicing summer season from seasonalmerge
ssp_ESM4_Summer_szn= ssp_esm4_seasonalmerge.sel(season='DJF') #MUST USE SEASONAL MERGE NOT SPEED SEASONAL MERGE. BE CAREFUL

## slicing the winter speed
ssp_ESM4_winter_szn_speed = ssp_esm4_SPEED_seasonalmerge.sel(season=slice('JJA')).mean(dim='season')
## Slicing winter season from seasonalmerge
ssp_ESM4_winter_szn= ssp_esm4_seasonalmerge.sel(season='JJA') #MUST USE SEASONAL MERGE NOT SPEED SEASONAL MERGE. BE CAREFUL

## slicing the fall speed
ssp_ESM4_fall_szn_speed = ssp_esm4_SPEED_seasonalmerge.sel(season=slice('MAM')).mean(dim='season')
## Slicing fall season from seasonalmerge
ssp_ESM4_fall_szn= ssp_esm4_seasonalmerge.sel(season='MAM') #MUST USE SEASONAL MERGE NOT SPEED SEASONAL MERGE. BE CAREFUL

## slicing the spring speed
ssp_ESM4_spring_szn_speed = ssp_esm4_SPEED_seasonalmerge.sel(season=slice('SON')).mean(dim='season')
## Slicing spring season from seasonalmerge
ssp_ESM4_spring_szn= ssp_esm4_seasonalmerge.sel(season='SON') #MUST USE SEASONAL MERGE NOT SPEED SEASONAL MERGE. BE CAREFUL

In [61]:
lat_esm4_ssp = ssp_esm4_speed_merge.coords['lat'].values #1-D numpy array of model's latitude values
lon_esm4_ssp = ssp_esm4_speed_merge.coords['lon'].values #1-D numpy array of model's longitude values
# make sure to replace the name of the data array if you chose a
# different name and use the appropriate coordinate names within the ['']
# based on the coordinates of lat / lon in your dataset.
# Pass the arrays to the standard_grid_cell_area function
esm4_ssp_areacell = standard_grid_cell_area(lat_esm4_ssp,lon_esm4_ssp)

# The areacella_xarray function outputs a numpy array. Convert this back into
# and xarray DataArray for further use. If this is a field you will be using
# often. You may wish to save this newly created DataArray as a Dataset and save
# as a netcdf file.
# here we assign dimensions and coordinates that are the same as our original SST array:
areacell_esm4_ssp = xr.DataArray(esm4_his_areacell,dims=({'lat':180, 'lon':360}),\
                                                coords=(ssp_esm4_speed_merge.coords['lat'],\
                                                        ssp_esm4_speed_merge.coords['lon']))

In [62]:
weights_esm4_ssp=areacell_esm4_ssp

In [63]:
weight_ross_ESM4_ssp=(mask_ESM4.ross_ESM4*ssp_esm4_speed_merge).weighted(weights_esm4_ssp).mean(dim=['lat','lon'])
weight_filchner_ESM4_ssp=(mask_ESM4.filchner_ESM4*ssp_esm4_speed_merge).weighted(weights_esm4_ssp).mean(dim=['lat','lon'])
weight_peninsula_ESM4_ssp= (mask_ESM4.peninsula_ESM4*ssp_esm4_speed_merge).weighted(weights_esm4_ssp).mean(dim=['lat','lon'])
weight_loweast_ESM4_ssp= (mask_ESM4.loweast_ESM4*ssp_esm4_speed_merge).weighted(weights_esm4_ssp).mean(dim=['lat','lon'])
weight_lowwest_ESM4_ssp= (mask_ESM4.lowwest_ESM4*ssp_esm4_speed_merge).weighted(weights_esm4_ssp).mean(dim=['lat','lon'])
weight_plateau_ESM4_ssp= (mask_ESM4.plateau_ESM4*ssp_esm4_speed_merge).weighted(weights_esm4_ssp).mean(dim=['lat','lon'])

In [64]:
weight_ross_ESM4_annual_ssp = ml.util.annual_average(weight_ross_ESM4_ssp,tcoord='time')
weight_filchner_ESM4_annual_ssp= ml.util.annual_average(weight_filchner_ESM4_ssp, tcoord='time')
weight_peninsula_ESM4_annual_ssp=ml.util.annual_average(weight_peninsula_ESM4_ssp, tcoord='time')
weight_loweast_ESM4_annual_ssp=ml.util.annual_average(weight_loweast_ESM4_ssp, tcoord='time')
weight_lowwest_ESM4_annual_ssp=ml.util.annual_average(weight_lowwest_ESM4_ssp, tcoord='time')
weight_plateau_ESM4_annual_ssp=ml.util.annual_average(weight_plateau_ESM4_ssp, tcoord='time')

In [65]:
ESM4_ssp_ross_wind=weight_ross_ESM4_annual_ssp.mean(dim='time')
ESM4_ssp_filch_wind=weight_filchner_ESM4_annual_ssp.mean(dim='time')
ESM4_ssp_pen_wind=weight_peninsula_ESM4_annual_ssp.mean(dim='time')
ESM4_ssp_east_wind=weight_loweast_ESM4_annual_ssp.mean(dim='time')
ESM4_ssp_west_wind=weight_lowwest_ESM4_annual_ssp.mean(dim='time')
ESM4_ssp_plat_wind=weight_plateau_ESM4_annual_ssp.mean(dim='time')

In [66]:
print(ESM4_ssp_ross_wind.values)
print(ESM4_ssp_filch_wind.values)
print(ESM4_ssp_pen_wind.values)
print(ESM4_ssp_east_wind.values)
print(ESM4_ssp_west_wind.values)
print(ESM4_ssp_plat_wind.values)

2.535717458676808
2.3090567137077205
3.2539406470074943
6.725562011401008
4.303066327873063
4.785620492882357


In [67]:
ross_total_std_ESM4_ssp= weight_ross_ESM4_annual_ssp.std('time')
filchner_total_std_ESM4_ssp= weight_filchner_ESM4_annual_ssp.std('time')
peninsula_total_std_ESM4_ssp=weight_peninsula_ESM4_annual_ssp.std('time')
loweast_total_std_ESM4_ssp=weight_loweast_ESM4_annual_ssp.std('time')
lowwest_total_std_ESM4_ssp=weight_lowwest_ESM4_annual_ssp.std('time')
plateau_total_std_ESM4_ssp=weight_plateau_ESM4_annual_ssp.std('time')

In [68]:
print(ross_total_std_ESM4_ssp.values)
print(filchner_total_std_ESM4_ssp.values)
print(peninsula_total_std_ESM4_ssp.values)
print(loweast_total_std_ESM4_ssp.values)
print(lowwest_total_std_ESM4_ssp.values)
print(plateau_total_std_ESM4_ssp.values)

0.1000144089935089
0.1248915851530563
0.13183640937986196
0.06691110632043928
0.09639411977809959
0.08259639956763354


# CM4 historical

In [69]:
##importing Eastward Near-Surface Wind data from google cloud. Amon
df_uas_historical = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
	
# step two … # replace with the id you need and other info to ween down datasets 
df_u_historical = df_uas_historical.query("source_id=='GFDL-CM4' & experiment_id == 'historical' & variable_id == 'uas'& table_id == 'Amon'")

# step three - loading in the data and storing it
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_u_historical.zstore.values[-1]
mapper = gcs.get_mapper(zstore)
ds_uas_historical_use = xr.open_zarr(mapper, consolidated=True)
Uas_historical = ds_uas_historical_use.uas #extracting uas variable
Uas_historical

<xarray.DataArray 'uas' (time: 1980, lat: 180, lon: 288)>
dask.array<open_dataset-uas, shape=(1980, 180, 288), dtype=float32, chunksize=(305, 180, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 ...
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * time     (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    interp_method:  conserve_order2
    long_name:      Eastward Near-Surface Wind
    original_name:  uas
    standard_name:  eastward_wind
    units:          m s-1

In [70]:
##importing Northward Near-Surface Wind data from google cloud.
df_vas_historical = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
	
# step two … # replace with the id you need and other info to ween down datasets 
df_v_historical = df_vas_historical.query("source_id=='GFDL-CM4' & experiment_id == 'historical' & variable_id == 'vas'& table_id == 'Amon'")

# step three 
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_v_historical.zstore.values[-1]
mapper = gcs.get_mapper(zstore)
ds_vas_historical = xr.open_zarr(mapper, consolidated=True)
historical_vas=ds_vas_historical.vas

In [71]:
##importing Northward Near-Surface Wind data from google cloud.
df_vas_historical = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
	
# step two … # replace with the id you need and other info to ween down datasets 
df_v_historical = df_vas_historical.query("source_id=='GFDL-CM4' & experiment_id == 'historical' & variable_id == 'vas'& table_id == 'Amon'")

# step three 
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_v_historical.zstore.values[-1]
mapper = gcs.get_mapper(zstore)
ds_vas_historical = xr.open_zarr(mapper, consolidated=True)
historical_vas=ds_vas_historical.vas

In [72]:
#slicing into seasons and averaging it, also selecting south pole region 
#time is averaged already so be careful when selecting time 
vas_his_seasonal=historical_vas.sel(time=slice('1995-01-16','2014-12-16')).groupby('time.season').mean(dim='time').sel(lat=slice(-89.5,-55))
vas_his_seasonal
uas_his_seasonal=Uas_historical.sel(time=slice('1995-01-16','2014-12-16')).groupby('time.season').mean(dim='time').sel(lat=slice(-89.5,-55))
uas_his_seasonal

<xarray.DataArray 'uas' (season: 4, lat: 35, lon: 288)>
dask.array<getitem, shape=(4, 35, 288), dtype=float32, chunksize=(1, 35, 288), chunktype=numpy.ndarray>
Coordinates:
    height   float64 ...
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 ... -58.5 -57.5 -56.5 -55.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    interp_method:  conserve_order2
    long_name:      Eastward Near-Surface Wind
    original_name:  uas
    standard_name:  eastward_wind
    units:          m s-1

In [73]:
## merge winds by merging eastward and northward.
#extracting just uas and vas data
Uas_historical1 = ds_uas_historical_use.uas.sel(lat=slice(-89.5,-55)).sel(time=slice('1995-01-16','2014-12-16'))
historical_vas1=ds_vas_historical.vas.sel(lat=slice(-89.5,-55)).sel(time=slice('1995-01-16','2014-12-16'))

his_mergewinds = xr.merge([Uas_historical1,historical_vas1]) # merge winds to make it easy
cm4_his_xmerge = his_mergewinds.lon #extracting the lon component fomr the mergewinds array
cm4_his_ymerge = his_mergewinds.lat
cm4_his_umerge = his_mergewinds.uas 
cm4_his_vmerge = his_mergewinds.vas
cm4_his_speedmerge = np.sqrt(np.square(cm4_his_umerge) + np.square(cm4_his_vmerge)) ## calculating speed using vector formula
## average
cm4_his_timeaveragemerge = his_mergewinds.sel(time=slice('1995-01-16','2014-12-16')).mean(dim='time')


### slicing the speed for the south pole
cm4_his_speedslice = cm4_his_speedmerge.mean(dim='time')
## UAS seasonal dataset

## merging the UAS seasonal and Vas seasonal to get merge seasonal
cm4_his_seasonal_merge = xr.merge([uas_his_seasonal,vas_his_seasonal])

In [74]:
## calculating the seasonal wind speed
cm4_his_seasonal_speedmerge = np.sqrt(np.square(cm4_his_seasonal_merge.uas) + np.square(cm4_his_seasonal_merge.vas))

## slicing the summer speed
cm4_his_Summer_szn_speed = cm4_his_seasonal_speedmerge.sel(season=slice('DJF')).mean(dim='season')
## Slicing summer season from seasonalmerge
cm4_his_Summer_szn= cm4_his_seasonal_merge.sel(season='DJF')

## slicing seasonalspeed to get speed for winter and averaging it
cm4_his_Winter_szn_speed = cm4_his_seasonal_speedmerge.sel(season=slice('JJA')).mean(dim='season')
cm4_his_Winter_szn= cm4_his_seasonal_merge.sel(season='JJA')

## FALL seasonal speed merge and averaging
cm4_his_Fall_szn_speed = cm4_his_seasonal_speedmerge.sel(season=slice('MAM')).mean(dim='season')
cm4_his_Fall_szn= cm4_his_seasonal_merge.sel(season='MAM')

## slicing seasonalspeed to get spring values and averaging it
cm4_his_Spring_szn_speed = cm4_his_seasonal_speedmerge.sel(season=slice('SON')).mean(dim='season')
cm4_his_Spring_szn= cm4_his_seasonal_merge.sel(season='SON')

In [75]:
lat_CM4_historical = cm4_his_speedmerge.coords['lat'].values #1-D numpy array of model's latitude values
lon_CM4_historical = cm4_his_speedmerge.coords['lon'].values #1-D numpy array of model's longitude values
# make sure to replace the name of the data array if you chose a
# different name and use the appropriate coordinate names within the ['']
# based on the coordinates of lat / lon in your dataset.
# Pass the arrays to the standard_grid_cell_area function
areacell = standard_grid_cell_area(lat_CM4_historical,lon_CM4_historical)

# The areacella_xarray function outputs a numpy array. Convert this back into
# and xarray DataArray for further use. If this is a field you will be using
# often. You may wish to save this newly created DataArray as a Dataset and save
# as a netcdf file.
# here we assign dimensions and coordinates that are the same as our original SST array:
areacell_CM4_historical = xr.DataArray(areacell,dims=({'lat':180, 'lon':360}),\
                                                coords=(cm4_his_speedmerge.coords['lat'],\
                                                        cm4_his_speedmerge.coords['lon']))

In [76]:
weights_cm4=areacell_CM4_historical

In [77]:
weight_ross_CM4_historical=(mask_CM4.ross_CM4*speedmerge).weighted(weights_cm4).mean(dim=['lat','lon'])
weight_filchner_CM4_historical=(mask_CM4.filchner_CM4*speedmerge).weighted(weights_cm4).mean(dim=['lat','lon'])
weight_peninsula_CM4_historical= (mask_CM4.peninsula_CM4*speedmerge).weighted(weights_cm4).mean(dim=['lat','lon'])
weight_loweast_CM4_historical= (mask_CM4.loweast_CM4*speedmerge).weighted(weights_cm4).mean(dim=['lat','lon'])
weight_lowwest_CM4_historical= (mask_CM4.lowwest_CM4*speedmerge).weighted(weights_cm4).mean(dim=['lat','lon'])
weight_plateau_CM4_historical= (mask_CM4.plateau_CM4*speedmerge).weighted(weights_cm4).mean(dim=['lat','lon'])

NameError: name 'speedmerge' is not defined

In [ ]:
weight_ross_cm4_annual_historical = ml.util.annual_average(weight_ross_CM4_historical,tcoord='time')
weight_filchner_cm4_annual_historical= ml.util.annual_average(weight_filchner_CM4_historical, tcoord='time')
weight_peninsula_cm4_annual_historical=ml.util.annual_average(weight_peninsula_CM4_historical, tcoord='time')
weight_loweast_cm4_annual_historical=ml.util.annual_average(weight_loweast_CM4_historical, tcoord='time')
weight_lowwest_cm4_annual_historical=ml.util.annual_average(weight_lowwest_CM4_historical, tcoord='time')
weight_plateau_cm4_annual_historical=ml.util.annual_average(weight_plateau_CM4_historical, tcoord='time')

In [ ]:
cm4_his_ross_wind=weight_ross_cm4_annual_historical.mean(dim='time')
cm4_his_filch_wind=weight_filchner_cm4_annual_historical.mean(dim='time')
cm4_his_pen_wind=weight_peninsula_cm4_annual_historical.mean(dim='time')
cm4_his_east_wind=weight_loweast_cm4_annual_historical.mean(dim='time')
cm4_his_west_wind=weight_lowwest_cm4_annual_historical.mean(dim='time')
cm4_his_plat_wind=weight_plateau_cm4_annual_historical.mean(dim='time')

In [ ]:
print(cm4_his_ross_wind.values)
print(cm4_his_filch_wind.values)
print(cm4_his_pen_wind.values)
print(cm4_his_east_wind.values)
print(cm4_his_west_wind.values)
print(cm4_his_plat_wind.values)

In [ ]:
ross_total_std_CM4= weight_ross_cm4_annual_historical.std('time')
filchner_total_std_CM4= weight_filchner_cm4_annual_historical.std('time')
peninsula_total_std_CM4=weight_peninsula_cm4_annual_historical.std('time')
loweast_total_std_CM4=weight_loweast_cm4_annual_historical.std('time')
lowwest_total_std_CM4=weight_lowwest_cm4_annual_historical.std('time')
plateau_total_std_CM4=weight_plateau_cm4_annual_historical.std('time')

In [ ]:
print(ross_total_std_CM4.values)
print(filchner_total_std_CM4.values)
print(peninsula_total_std_CM4.values)
print(loweast_total_std_CM4.values)
print(lowwest_total_std_CM4.values)
print(plateau_total_std_CM4.values)

# CM4 ssp585

In [ ]:
##importing Eastward Near-Surface Wind data from google cloud. Amon
df_uas_cm4_ssp = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')	
# step two … # replace with the id you need and other info to ween down datasets 
df_u_cm4_ssp = df_uas_cm4_ssp.query("source_id=='GFDL-CM4' & experiment_id == 'ssp585' & variable_id == 'uas'& table_id == 'Amon'")
# step three - loading in the data and storing it
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_u_cm4_ssp.zstore.values[-1]
mapper = gcs.get_mapper(zstore)
ds_uas_cm4_ssp = xr.open_zarr(mapper, consolidated=True)
ds_uas_cm4_ssp

In [ ]:
##importing Eastward Near-Surface Wind data from google cloud. Amon
df_vas_cm4_ssp = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

# step two … # replace with the id you need and other info to ween down datasets 
df_v_cm4_ssp =df_vas_cm4_ssp.query("source_id=='GFDL-CM4' & experiment_id == 'ssp585' & variable_id == 'vas'& table_id == 'Amon'")

# step three - loading in the data and storing it
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_v_cm4_ssp.zstore.values[-1]
mapper = gcs.get_mapper(zstore)
ds_vas_cm4_spp = xr.open_zarr(mapper, consolidated=True)
ds_vas_cm4_spp

In [ ]:
#defining variables that extract wind speed variables
cm4_ssp_uas=ds_uas_cm4_ssp.uas
cm4_ssp_vas=ds_vas_cm4_spp.vas

In [ ]:
#slicing into seasons and averaging it, also selecting south pole region 
#time is averaged already so be careful when selecting time 
cm4_ssp_seasonal_uas=ds_uas_cm4_ssp.uas.sel(time=slice('2081-01-16','2100-12-16')).groupby('time.season').mean(dim='time').sel(lat=slice(-89.5,-55))

cm4_ssp_seasonal_vas=ds_vas_cm4_spp.vas.sel(time=slice('2081-01-16','2100-12-16')).groupby('time.season').mean(dim='time').sel(lat=slice(-89.5,-55))
cm4_ssp_seasonal_vas

In [ ]:
cm4_ssp_mergewinds = xr.merge([cm4_ssp_uas,cm4_ssp_vas]) # merge winds to make it easy
cm4_ssp_xmerge = cm4_ssp_mergewinds.lon #extracting the lon component from the mergewinds array
cm4_ssp_ymerge = cm4_ssp_mergewinds.lat
cm4_ssp_umerge = cm4_ssp_mergewinds.uas 
cm4_ssp_vmerge = cm4_ssp_mergewinds.vas
cm4_ssp_speedmerge = np.sqrt(np.square(cm4_ssp_umerge) + np.square(cm4_ssp_vmerge)) ## calculating speed using vector formula

In [ ]:
## average time for quiver plots
cm4_ssp_timeaveragemerge = cm4_ssp_mergewinds.sel(time=slice('2081-01-16','2100-12-16')).mean(dim='time')

### the data that is used for overall regional speed
cm4_ssp_speedslice = cm4_ssp_speedmerge.mean(dim='time')

## merging the UAS seasonal and Vas seasonal to get seasonal merge
cm4_ssp_seasonalmerge = xr.merge([cm4_ssp_seasonal_uas,cm4_ssp_seasonal_vas])


In [ ]:
cm4_ssp_seasonalmerge

In [ ]:
## calculating the seasonal wind speed
cm4_ssp_seasonal_speedmerge = np.sqrt(np.square(cm4_ssp_seasonalmerge.uas) + np.square(cm4_ssp_seasonalmerge.vas))

## slicing the summer speed
cm4_ssp_Summer_szn_speed = cm4_ssp_seasonalmerge.sel(season=slice('DJF')).mean(dim='season')
## Slicing summer season from seasonalmerge
cm4_ssp_Summer_szn= cm4_ssp_seasonalmerge.sel(season='DJF')

## slicing seasonalspeed to get speed for winter and averaging it
cm4_ssp_Winter_szn_speed = cm4_ssp_seasonalmerge.sel(season=slice('JJA')).mean(dim='season')
cm4_ssp_Winter_szn= cm4_ssp_seasonalmerge.sel(season='JJA')

## FALL seasonal speed merge and averaging
cm4_ssp_Fall_szn_speed = cm4_ssp_seasonalmerge.sel(season=slice('MAM')).mean(dim='season')
cm4_ssp_Fall_szn= cm4_ssp_seasonalmerge.sel(season='MAM')

## slicing seasonalspeed to get spring values and averaging it
cm4_ssp_Spring_szn_speed = cm4_ssp_seasonalmerge.sel(season=slice('SON')).mean(dim='season')
cm4_ssp_Spring_szn= cm4_ssp_seasonalmerge.sel(season='SON')

In [ ]:
lat_CM4_ssp = cm4_ssp_speedmerge.coords['lat'].values #1-D numpy array of model's latitude values
lon_CM4_ssp = cm4_ssp_speedmerge.coords['lon'].values #1-D numpy array of model's longitude values
# make sure to replace the name of the data array if you chose a
# different name and use the appropriate coordinate names within the ['']
# based on the coordinates of lat / lon in your dataset.
# Pass the arrays to the standard_grid_cell_area function
cm4_ssp_areacell = standard_grid_cell_area(lat_CM4_ssp,lon_CM4_ssp)

# The areacella_xarray function outputs a numpy array. Convert this back into
# and xarray DataArray for further use. If this is a field you will be using
# often. You may wish to save this newly created DataArray as a Dataset and save
# as a netcdf file.
# here we assign dimensions and coordinates that are the same as our original SST array:
areacell_CM4_ssp = xr.DataArray(cm4_ssp_areacell,dims=({'lat':180, 'lon':360}),\
                                                coords=(cm4_ssp_speedmerge.coords['lat'],\
                                                        cm4_ssp_speedmerge.coords['lon']))

In [ ]:
cm4_ssp_weights=areacell_CM4_ssp

In [ ]:
weight_ross_CM4_ssp=(mask_CM4.ross_CM4*cm4_ssp_speedmerge).weighted(cm4_ssp_weights).mean(dim=['lat','lon'])
weight_filchner_CM4_ssp=(mask_CM4.filchner_CM4*cm4_ssp_speedmerge).weighted(cm4_ssp_weights).mean(dim=['lat','lon'])
weight_peninsula_CM4_ssp= (mask_CM4.peninsula_CM4*cm4_ssp_speedmerge).weighted(cm4_ssp_weights).mean(dim=['lat','lon'])
weight_loweast_CM4_ssp= (mask_CM4.loweast_CM4*cm4_ssp_speedmerge).weighted(cm4_ssp_weights).mean(dim=['lat','lon'])
weight_lowwest_CM4_ssp= (mask_CM4.lowwest_CM4*cm4_ssp_speedmerge).weighted(cm4_ssp_weights).mean(dim=['lat','lon'])
weight_plateau_CM4_ssp= (mask_CM4.plateau_CM4*cm4_ssp_speedmerge).weighted(cm4_ssp_weights).mean(dim=['lat','lon'])

In [ ]:
weight_ross_cm4_annual_ssp = ml.util.annual_average(weight_ross_CM4_ssp,tcoord='time')
weight_filchner_cm4_annual_ssp= ml.util.annual_average(weight_filchner_CM4_ssp, tcoord='time')
weight_peninsula_cm4_annual_ssp=ml.util.annual_average(weight_peninsula_CM4_ssp, tcoord='time')
weight_loweast_cm4_annual_ssp=ml.util.annual_average(weight_loweast_CM4_ssp, tcoord='time')
weight_lowwest_cm4_annual_ssp=ml.util.annual_average(weight_lowwest_CM4_ssp, tcoord='time')
weight_plateau_cm4_annual_ssp=ml.util.annual_average(weight_plateau_CM4_ssp, tcoord='time')

In [ ]:
cm4_ssp_ross_wind=weight_ross_cm4_annual_ssp.mean(dim='time')
cm4_ssp_filch_wind=weight_filchner_cm4_annual_ssp.mean(dim='time')
cm4_ssp_pen_wind=weight_peninsula_cm4_annual_ssp.mean(dim='time')
cm4_ssp_east_wind=weight_loweast_cm4_annual_ssp.mean(dim='time')
cm4_ssp_west_wind=weight_lowwest_cm4_annual_ssp.mean(dim='time')
cm4_ssp_plat_wind=weight_plateau_cm4_annual_ssp.mean(dim='time')

In [ ]:
print(cm4_ssp_ross_wind.values)
print(cm4_ssp_filch_wind.values)
print(cm4_ssp_pen_wind.values)
print(cm4_ssp_east_wind.values)
print(cm4_ssp_west_wind.values)
print(cm4_ssp_plat_wind.values)

In [ ]:
ross_total_std_CM4_ssp= weight_ross_cm4_annual_ssp.std('time')
filchner_total_std_CM4_ssp= weight_filchner_cm4_annual_ssp.std('time')
peninsula_total_std_CM4_ssp=weight_peninsula_cm4_annual_ssp.std('time')
loweast_total_std_CM4_ssp=weight_loweast_cm4_annual_ssp.std('time')
lowwest_total_std_CM4_ssp=weight_lowwest_cm4_annual_ssp.std('time')
plateau_total_std_CM4_ssp=weight_plateau_cm4_annual_ssp.std('time')

In [ ]:
print(ross_total_std_CM4_ssp.values)
print(filchner_total_std_CM4_ssp.values)
print(peninsula_total_std_CM4_ssp.values)
print(loweast_total_std_CM4_ssp.values)
print(lowwest_total_std_CM4_ssp.values)
print(plateau_total_std_CM4_ssp.values)

# ERA5 

In [ ]:
era5_vas=xr.open_dataset(('/home/tu.temple.edu/tuo15329/era5_wind/era5_10m_v_component_of_wind_1940-2024_monthly_ensemble.nc'),use_cftime=True)

In [ ]:
era5_vas_noleap=era5_vas.assign_attrs(calendar='noleap')
era5_vas_use =era5_vas_noleap.mean(dim='expver')
era5_vas_use

In [ ]:
ERA5_vas_95_2014=era5_vas_use.v10.mean(dim='number').sel(time=slice('1995-01-01','2014-12-01'))
ERA5_vas_95_2014

In [ ]:
era5_uas=xr.open_dataset(('/home/tu.temple.edu/tuo15329/era5_wind/era5_10m_u_component_of_wind_1940-2024_monthly_ensemble.nc'),use_cftime=True)
era5_uas_noleap=era5_uas.assign_attrs(calendar='noleap')
era5_uas_use =era5_uas_noleap.mean('expver')#sel(expver=1).combine_first(era5_uas.sel(expver=5))
#ERA5_combine_uas.load()
#era5_uas_use=ERA5_combine_uas.u10.mean(dim='number')
#era5_uas_use
era5_uas_use

In [ ]:
ERA5_uas_95_2014=era5_uas_use.sel(time=slice('1995-01-01','2014-12-01'))
ERA5_uas_95_2014

In [ ]:
#era5 wind components 
ERA5_vas_95_2014
ERA5_uas_95_2014

In [ ]:
ERA5_merge_winds=xr.merge([ERA5_vas_95_2014,ERA5_uas_95_2014])
xmerge = ERA5_merge_winds.longitude #extracting the lon component fomr the mergewinds array
ymerge = ERA5_merge_winds.latitude
umerge = ERA5_merge_winds.u10
vmerge = ERA5_merge_winds.v10
speedmerge = np.sqrt(np.square(umerge) + np.square(vmerge)) ## calculating speed using vector sum formula
## average
timeaveragemerge = ERA5_merge_winds.sel(time=slice('1995-01-01','2014-12-01')).mean(dim='time')
timeaveragemerge

In [ ]:
#was not getting one value for the std so meaned out number and this fixed it
test=speedmerge.mean(dim='number')
test

In [ ]:
lat_ERA5 = test.coords['latitude'].values #1-D numpy array of model's latitude values
lon_ERA5 = test.coords['longitude'].values #1-D numpy array of model's longitude values

# make sure to replace the name of the data array if you chose a 
# different name and use the appropriate coordinate names within the ['']
# based on the coordinates of lat / lon in your dataset.

# Pass the arrays to the standard_grid_cell_area function
areacell = standard_grid_cell_area(lat_ERA5,lon_ERA5)  

# The areacella_xarray function outputs a numpy array. Convert this back into 
# and xarray DataArray for further use. If this is a field you will be using 
# often. You may wish to save this newly created DataArray as a Dataset and save 
# as a netcdf file.

# here we assign dimensions and coordinates that are the same as our original SST array:

areacell_ERA5 = xr.DataArray(areacell,dims=({'latitude':180, 'longitude':360}),\
                                                coords=(test.coords['latitude'],\
                                                        test.coords['longitude']))

In [ ]:
weights_ERA5_wind=areacell_ERA5

In [ ]:
weight_ross_ERA5=(mask_ERA5.ross_ERA5*test).weighted(weights_ERA5_wind).mean(dim=['latitude','longitude'])
weight_filchner_ERA5=(mask_ERA5.filchner_ERA5*test).weighted(weights_ERA5_wind).mean(dim=['latitude','longitude'])
weight_peninsula_ERA5= (mask_ERA5.peninsula_ERA5*test).weighted(weights_ERA5_wind).mean(dim=['latitude','longitude'])
weight_loweast_ERA5= (mask_ERA5.loweast_ERA5*test).weighted(weights_ERA5_wind).mean(dim=['latitude','longitude'])
weight_lowwest_ERA5= (mask_ERA5.lowwest_ERA5*test).weighted(weights_ERA5_wind).mean(dim=['latitude','longitude'])
weight_plateau_ERA5= (mask_ERA5.plateau_ERA5*test).weighted(weights_ERA5_wind).mean(dim=['latitude','longitude'])

In [ ]:
weight_ross_ERA5_annual_wind = ml.util.annual_average(weight_ross_ERA5, tcoord='time')
weight_filchner_ERA5_annual_wind= ml.util.annual_average(weight_filchner_ERA5, tcoord='time')
weight_peninsula_ERA5_annual_wind=ml.util.annual_average(weight_peninsula_ERA5, tcoord='time')
weight_loweast_ERA5_annual_wind=ml.util.annual_average(weight_loweast_ERA5, tcoord='time')
weight_lowwest_ERA5_annual_wind=ml.util.annual_average(weight_lowwest_ERA5, tcoord='time')
weight_plateau_ERA5_annual_wind=ml.util.annual_average(weight_plateau_ERA5, tcoord='time')

In [ ]:
ross_wind=weight_ross_ERA5_annual_wind.mean(dim='time')
filch_wind=weight_filchner_ERA5_annual_wind.mean(dim='time')
pen_wind=weight_peninsula_ERA5_annual_wind.mean(dim='time')
east_wind=weight_loweast_ERA5_annual_wind.mean(dim='time')
west_wind=weight_lowwest_ERA5_annual_wind.mean(dim='time')
plat_wind=weight_plateau_ERA5_annual_wind.mean(dim='time')

In [ ]:
print(ross_wind.values)
print(filch_wind.values)
print(pen_wind.values)
print(east_wind.values)
print(west_wind.values)
print(plat_wind.values)

In [ ]:
ross_total_std_ERA5= weight_ross_ERA5_annual_wind.std('time')
filchner_total_std_ERA5= weight_filchner_ERA5_annual_wind.std('time')
peninsula_total_std_ERA5=weight_peninsula_ERA5_annual_wind.std('time')
loweast_total_std_ERA5=weight_loweast_ERA5_annual_wind.std('time')
lowwest_total_std_ERA5=weight_lowwest_ERA5_annual_wind.std('time')
plateau_total_std_ERA5=weight_plateau_ERA5_annual_wind.std('time')

In [ ]:
print(ross_total_std_ERA5.values)
print(filchner_total_std_ERA5.values)
print(peninsula_total_std_ERA5.values)
print(loweast_total_std_ERA5.values)
print(lowwest_total_std_ERA5.values)
print(plateau_total_std_ERA5.values)

In [ ]:
fig, ax= plt.subplots(figsize= (15,10))

X = ['Ross','Filchner','Peninsula','Low East', 'Low West', 'Plateau'] 
CM4_total = [251.69094432376951,
247.8710239334481,
263.8351310124591,
247.23427191545653,
246.81944006808382,
227.04537479287538] 

CM4_SSP585_total=[255.36364887913763,
253.27195864068844,
267.69436623114564,
250.35929333353494,
250.7222091651273,
231.02818575888872]

ESM4_total = [250.54319814468258,
247.03323504519855,
263.2886219953197,
246.9820592721955,
246.6824680168488,
226.91860301842905] 

ESM4_SSP585_total=[254.07582417183127,
252.5920194094377,
266.8595396549488,
250.28772172473833,
250.40099611680156,
231.22473522844024]

ERA5_total = [250.21906368696284,
248.7180793419007,
263.1056430905821,
246.9384614212835,
247.21904361867243,
229.76085589795244]

CM4X_total = [248.55057734177527,
244.77431073904046,
259.60761241057855,
245.64900287020993,
244.5743954791298,
225.76823331693862]

CM4X_SSP585_total=[253.04681481774344,
250.51421552673054,
264.7410907063711,
249.4837195434456,
248.72956238537125,
230.26246188075638]

std_cm4=[0.9910555828414944,
0.9893999351184022,
0.5514007507967089,
0.447637454248452,
0.5079859574142839,
0.5043536355732328]

std_esm4=[1.0730124903358855,
0.9218497080222389,
0.6767918332642133,
0.5322224702646161,
0.3558740663919748,
0.6114135755899789]

std_era5= [1.0730124903358855,
0.9218497080222389,
0.6767918332642133,
0.5322224702646161,
0.3558740663919748,
0.6114135755899789]

std_cm4x = [0.8326426732367908,
0.8231899134493114,
0.9651506687485614,
0.43920573487175796,
0.42473732258805924,
0.4653856639934059]

std_cm4x_ssp585= [1.3135812787599368,
0.8258963356231306,
0.662054067960358,
0.6293363855071491,
0.7337344177371089,
0.7573066044121799]

std_cm4_ssp585=[0.8811322909185608,
1.0546983927577607,
0.5807812239416166,
0.7056089730955786,
0.6829191335782888,
0.7399788918517587]

std_esm4_ssp585=[1.2799813627771934,
0.7401616875632971,
0.7955785216484689,
0.4440107557249149,
0.612012509579448,
0.5234501239664328]


X_axis = np.arange(len(X)) 

plt.bar(X_axis-0.2, ESM4_total, 0.1, label = 'ESM4 Historical 1995-2014', color='r',edgecolor='black', alpha=0.5) 
plt.errorbar(X_axis-0.2, ESM4_total,yerr= std_esm4,fmt='o', color= 'black', markersize=3, capsize=4)

plt.bar(X_axis-0.1, ESM4_SSP585_total, 0.1, label= 'ESM4 SSP585 2081-2100', color ='r', edgecolor='black', alpha=0.5, hatch='///')
plt.errorbar(X_axis-0.1, ESM4_SSP585_total, yerr= std_esm4_ssp585, fmt='o', color='black', markersize=3, capsize=4)

plt.bar(X_axis, CM4_total, 0.1, label = 'CM4 Historical 1995-2014', color= 'blue',edgecolor='black', alpha=0.5) 
plt.errorbar(X_axis, CM4_total, yerr=std_cm4, fmt='o', color='black', markersize=3, capsize=4)

plt.bar(X_axis+0.1,CM4_SSP585_total, 0.1, label='CM4 SSP585 2081-2100', color='blue', edgecolor='black', alpha=0.5, hatch='///')
plt.errorbar(X_axis+0.1, CM4_SSP585_total, yerr= std_cm4_ssp585, fmt='o', color='black', markersize=3, capsize=4)

plt.bar(X_axis+0.2, CM4X_total, 0.1, label= 'CM4X-p25 Historical 1995-2014', color='green', alpha=0.5,edgecolor='black')
plt.errorbar(X_axis+0.2, CM4X_total, yerr=std_cm4x, fmt= 'o', color='black', markersize=3, capsize=4)

plt.bar(X_axis+0.3, CM4X_SSP585_total, 0.1, label='CM4X-p25 SSP585 2081-2100', color='green', edgecolor='black', alpha=0.5, hatch='///')
plt.errorbar(X_axis+0.3, CM4X_SSP585_total, yerr= std_cm4x_ssp585, fmt='o', color='black', markersize=3, capsize=4)

plt.bar(X_axis + 0.4, ERA5_total, 0.1, label= 'ERA5 1995-2014', color= 'k', edgecolor='black', alpha=0.5)
plt.errorbar(X_axis+0.4, ERA5_total, yerr=std_era5, fmt='o', color='black', markersize=3, capsize=4)



ax.grid(linewidth=0.5, linestyle= '--')
ax.set_axisbelow(True)
plt.xticks(X_axis, X, fontsize=14) 
plt.xlabel("Region", fontsize=20) 
plt.ylabel("Wind Speed (m/s)", fontsize=20 )
plt.title("Wind Speed for each Region Weighted", fontsize=20) 
plt.legend(fontsize=15) 
ax.minorticks_on()
plt.ylim(220,270)
plt.savefig('TemperatureHistoricalWeighted.png')